In [4]:
from sklearn.cluster import KMeans #ML 
import pandas as pd #dados
import numpy as np #manipulacao de vetores
from matplotlib import pyplot as plt #plotar os gráficos
import pickle

# leitura dos dados
df = pd.read_csv(r"C:\Users\Leandro\Dropbox\00-PD\teste\Respostas\Questao 04\dados_premium.csv")
#df.head()
#df.info()


In [5]:
# divide o conjunto de dados em partes iguais e associa os dados em 4 grupos distintos para cada atributo
t_labels = range(1, 5)
t_groups = pd.qcut(df.DiasSemUso.rank(method='first'), q = 4, labels = t_labels)
e_labels = range(1, 5)
e_groups = pd.qcut(df.TotalSessoes.rank(method='first'), q = 4, labels = e_labels)
f_labels = range(1, 5)
f_groups = pd.qcut(df.TotalGasto.rank(method='first'), q = 4, labels = f_labels)

#print(t_labels)
#print(e_labels)
#print(f_labels)

#print(t_groups)
#print(e_groups)
#print(f_groups)

In [6]:
# cria uma nova coluna no DataFrame para cada valor (R, F e M)
df['R'] = t_groups.values
df['F'] = e_groups.values
df['M'] = f_groups.values

# Define os segmentos somdando os valores e estabele um score de classificacao para cada usuario
df['RFM_Segment'] = df.apply(lambda x: str(x['R']) + str(x['F']) + str(x['M']), axis = 1)
df['RFM_Score'] = df[['R', 'F', 'M']].sum(axis = 1)
#df.head()
#df.info()

In [7]:
# assign labels from total score
categoria_score = ['Green', 'Bronze', 'Silver', 'Gold']
grupos_score = pd.qcut(df.RFM_Score, q = 4, labels = categoria_score)
df['RFM_Level'] = grupos_score.values
df.head()
#df.info()

,StudentId,DiasSemUso,TotalSessoes,TotalGasto,R,F,M,RFM_Segment,RFM_Score,RFM_Level
0,39981208,0,6,29.9,1,1,1,111,3.0,Green
1,30437941,0,47,59.8,1,3,2,132,6.0,Green
2,49298868,75,49,29.9,4,3,1,431,8.0,Bronze
3,83170423,0,82,209.3,1,4,4,144,9.0,Silver
4,87293262,0,76,209.3,1,4,4,144,9.0,Silver


In [8]:
# agrupa os clientes nos clusters e calcula a média de cada cluster 
clusters_agrupados = df.groupby('RFM_Level').agg({
    'DiasSemUso' : 'mean',
    'TotalSessoes' : 'mean',
    'TotalGasto' : 'mean',
    'StudentId': 'count'
})

#renomar a coluna final
clusters_agrupados = clusters_agrupados.rename(columns={"StudentId": "Qtd. Clientes"}) 

clusters_agrupados.round(1).head()

,DiasSemUso,TotalSessoes,TotalGasto,Qtd. Clientes
RFM_Level,,,,
Green,16.8,30.4,38.8,499
Bronze,37.8,38.5,86.1,692
Silver,48.0,54.2,149.5,299
Gold,42.9,73.9,164.3,200


In [9]:
df.to_csv(r'C:\Users\Leandro\Dropbox\00-PD\teste\Respostas\Questao 04\resultadoRFMTradicional.csv', index = False)